In [19]:
from search.Problem import Problem
from search.Search import aStarSearch, breadthFirstSearch, ipa_star

ImportError: cannot import name 'breadthFirstSearch' from 'search.Search' (/Users/gabrielnativel-fontaine/Documents/Cours/Algorithmes de recherche/tp-ameliore/search/Search.py)

# L'algorithme A*
A* est un algorithme de recherche permettant de trouver le chemin dans un graphe entre un noeud de départ et un noeud d'arrivé.
Cette algorithme repose sur deux éléments importants:
 - Un problème formalisé
 - Une heuristique admissible et consistante
 
L'algorithme consiste à explorer les noeuds du graphes en passant par les meilleurs noeuds, que l'on classe selon leur valeur $f(n) = g(n) + h(n)$ où $g(n)$ est la profondeur du noeud $n$ et $h(n)$ l'heuristique au noeud $n$.

## Formalisation du problème
Il est nécessaire de réfléchir à la formalisation du problème que l'on souhaite résoudre.

### Le taquin
lorem ipsum...

### Un problème fantastique
...



In [ ]:
class UnProblem(Problem):
    def __init__(self, start, end):
        super().__init__(start, end)
        
    def _buildNext(self, prev):
        pass
    

## Heuristique
Une heuristique est une fonction qui, pour un noeud donné, donne le coût permettant d'atteindre le noeud but.

On dit d'une heuristique qu'elle est **admissible** lorsqu'elle est inférieure ou égale au coût réel pour se rendre au but.
On dit également d'une heuristique qu'elle est **consistante** (ou monotone) lorsque, pour chaque nœud n et chaque successeur n’ de n produit par n’importe quelle action, le coût estimé pour atteindre le but à partir de n n’est pas supérieur au coût de l’étape pour aller à n’ plus le coût estimé pour atteindre le but à partir de n’.

Tout l'art des algorithmes de recherche réside dans le développement d'heuristiques admissibles et consistantes afin de préserver l'optimalité de l'algorithme.

L'utilisation de certaines heuristiques dites triviales équivaut à l'utilisation d'algorithmes de recherches dérivés:
- Si $f(n) = g(n)$: algorithme du coût uniforme, qui équivaut à une exploration en largeur d'abord lorsque les coûts pour passer d'un noeud parent à un noeud fils sont égaux (Breadth First Search)
- Si $h(n) = 0 \Rightarrow f(n) = g(n)$: algorithme glouton (Best first search)

Créons une heuristique pour notre problème :

In [3]:
def heuristic(state):
    pass

## Résolution
Voyons le résultat de notre algorithme

In [ ]:
problem = UnProblem('debut', 'fin')
result = aStarSearch(problemTaquin, heuristic)
print(result)

## Une autre heuristique ?

In [16]:
def heuristicAutre(state):
    pass

In [ ]:
result2 = aStarSearch(problemTaquin, heuristicAutre)
print(result2)

In [ ]:
# Afficher le comparatif entre les deux heuristiques

## Comparaison avec d'autres algorithmes
### Recherche en largeur d'abord

In [ ]:
result_bfs = breadthFirstSeach(problemTaquin, heuristicAutre)
print(result_bfs)

### Recherche avec coût uniforme

In [ ]:
result_uc = uniformCostSeach(problemTaquin, heuristicAutre)
print(result_uc)

## Aller plus loin avec A*
### Step A*

In [ ]:
steps = []
final_path = []

start = ''
for step in steps:
    tmp_problem = UnProblem(start, step)
    tmp_result = aStarSearch(problemTaquin, heuristicAutre)
    
    final_path.extends(tmp_result.Path)
    start = step

### IPA*

In [ ]:
result_ipa = ipaStarSeach(problemTaquin, heuristicAutre)
print(result_ipa)